# Onboarding Agent

In [ ]:
import asyncio
from crewai import Agent, Crew, Task, Process
from crewai.tools import BaseTool
from pydantic import Field, BaseModel
from typing import Type
import re
import os

# Set up API key
os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY"

### Custom Tool

In [2]:
class EmailGeneratorInput(BaseModel):
    fullname: str = Field(..., description="The full name of the employee.")
    department: str = Field(..., description="The department of the employee.")

# --- Custom Tool Definition ---
class EmailGeneratorTool(BaseTool):
    name: str = "Email Generator"
    description: str = "Generates a company email based on the employee's full name and department."
    args_schema: Type[BaseModel] = EmailGeneratorInput

    def _run(self, fullname: str, department: str) -> str:
            """Generate a company email from fullname and department."""
            name_part = re.sub(r'[^a-zA-Z]', '', fullname.replace(' ', '.')).lower()
            department_part = department.replace(' ', '').lower()
            return f"{name_part}@{department_part}.company.com"

### Agents

In [3]:
scheduler_agent = Agent(
    role="Scheduler",
    goal="Validate and confirm employee start date",
    backstory="Ensures the selected start date is valid and not a holiday."
)

it_agent = Agent(
    role="IT Administrator",
    goal="Set up company email and provide PC setup instructions",
    backstory="Prepares company email accounts and delivers simple PC setup guidelines.",
    tools=[EmailGeneratorTool()]
)

report_agent = Agent(
    role="Onboarding Reporter",
    goal="Generate an onboarding summary report",
    backstory="Creates a final onboarding report with all key details for HR records."
)

### Tasks

In [4]:
validate_start_date_task = Task(
    description=(
        "Check if the start date ({employe}) is a working day. "
        "If it's a holiday or weekend, suggest the next available working day."
    ),
    expected_output="Confirmed valid start date or suggestion for a new date.",
    agent=scheduler_agent
)

it_setup_task = Task(
    description=(
        "Create a company email for {employe} in the {employe} department. "
        "Provide a short, clear set of bullet points on how to set up their PC."
    ),

    expected_output="Active company email and clear PC setup instructions.",
    agent=it_agent
)

generate_report_task = Task(
    description=(
        "Generate a summary onboarding report for {employe}, starting on {employe}, "
        "in the {employe} department. Include email details, PC setup info, and start date confirmation."
    ),
    expected_output="A clear and concise onboarding summary report.",
    agent=report_agent
)


### Crew

In [5]:
# --- Crew Definition ---
onboarding_crew = Crew(
    agents=[scheduler_agent, it_agent, report_agent],
    tasks=[validate_start_date_task, it_setup_task, generate_report_task],
    process=Process.sequential,
    verbose=True
)

### Asyncronous Function

In [8]:
async def async_onboarding_execution(onboarding_inputs):
    results = await onboarding_crew.kickoff_async(inputs=onboarding_inputs)
    print("Crew result:", results.raw)

### Main

In [9]:
import nest_asyncio
nest_asyncio.apply()


onboarding_inputs = {
    "employe": [
        "Alice Johnson, starting 2024-07-15, department Engineering",
        "Bob Smith, starting 2024-07-16, department Marketing",
        "Charlie Brown, starting 2024-07-17, department Sales"
    ]
}

# Run asynchronous onboarding workflows
asyncio.run(async_onboarding_execution(onboarding_inputs))

# Agent: Scheduler
## Task: Check if the start date (['Alice Johnson, starting 2024-07-15, department Engineering', 'Bob Smith, starting 2024-07-16, department Marketing', 'Charlie Brown, starting 2024-07-17, department Sales']) is a working day. If it's a holiday or weekend, suggest the next available working day.


# Agent: Scheduler
## Final Answer: 
Upon reviewing the proposed start dates, the following validations are made:

1. **Alice Johnson**: Starting on **2024-07-15** (Monday) - This date is a valid working day.
2. **Bob Smith**: Starting on **2024-07-16** (Tuesday) - This date is also a valid working day.
3. **Charlie Brown**: Starting on **2024-07-17** (Wednesday) - This date is a valid working day as well.

None of the proposed start dates fall on a holiday or weekend. Therefore, all employees can start on their originally scheduled dates without any modifications. 

Confirmed valid start dates:
- Alice Johnson: **2024-07-15**
- Bob Smith: **2024-07-16**
- Charlie Brown: *